# Trabalho Final de SGBD

Importando os recursos necessários.

In [72]:
import pandas as pd
import numpy as np

Lendo o arquivo CSS.

In [73]:
df = pd.read_csv('sinasc_nascimentos.csv')

C:\Users\Acer\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (21) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [74]:
# Algumas colunas não serão analisadas.
df = df.drop(['data_extracao', 'cbo_mae'], axis = 1)
df.info();

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190612 entries, 0 to 190611
Data columns (total 20 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   data_nascimento         190612 non-null  object
 1   escolaridade_mae        190532 non-null  object
 2   raca_mae                187450 non-null  object
 3   estado_civil_mae        190553 non-null  object
 4   nascimento_mae          187457 non-null  object
 5   ocupacao                190433 non-null  object
 6   nro_consultas_prenatal  190612 non-null  object
 7   cesaria_antes_parto     94676 non-null   object
 8   duracao_gestacao        190504 non-null  object
 9   gravidez                190582 non-null  object
 10  local_nascimento        190612 non-null  object
 11  parto                   190594 non-null  object
 12  trabalho_parto          186804 non-null  object
 13  apresentacao_crianca    187183 non-null  object
 14  classe_robson           21475 non-nu

### Etapa de pré-processamento

In [75]:
from sklearn.model_selection import train_test_split

# Separando os conjuntos de dados de treino e teste.
df_treino, df_teste = train_test_split(df, test_size = 0.2, random_state = 42)

# Separando a coluna alvo do conjunto de treino.
df_treino_labels = df_treino['possui_anomalia'].copy()
df_treino = df_treino.drop(columns = 'possui_anomalia')

# Separando a coluna alvo do conjunto de teste.
df_teste_labels = df_teste['possui_anomalia'].copy()
df_teste = df_teste.drop(columns = 'possui_anomalia')

In [76]:
# Imports necessários para o pré-processamento.
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [77]:
# Classificando os atributos como numéricos, categóricos ou binários.
nomes_atributos_numericos = ['peso']
nomes_atributos_categoricos = ['data_nascimento', 'escolaridade_mae', 'raca_mae', 'estado_civil_mae', 'nascimento_mae', 'ocupacao', 'nro_consultas_prenatal', 'cesaria_antes_parto', 'duracao_gestacao', 'gravidez', 'local_nascimento', 'parto', 'trabalho_parto', 'apresentacao_crianca', 'classe_robson', 'cid_anomalia', 'sexo', 'raca']

In [78]:
# Lidando com dados ausentes.
pipeline_atr_numericos = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
])

preproc_completo = ColumnTransformer([
    ('numericos', pipeline_atr_numericos, nomes_atributos_numericos),
    ('categoricos', OneHotEncoder(), nomes_atributos_categoricos),
], sparse_threshold = 0)

In [79]:
# Pré-processamento do conjunto de treino.
X_treino = preproc_completo.fit_transform(df_treino)

# Pré-processamento do conjunto de teste.
X_teste = preproc_completo.transform(df_teste)

# Coluna alvo para os conjuntos de treino e teste.
y_treino = df_treino_labels.values
y_teste = df_teste_labels.values

### Predição por Árvores de Decisão

In [ ]:
# Importa a árvore de decisão.
from sklearn.tree import DecisionTreeClassifier

arvore = DecisionTreeClassifier(max_depth = 5)
arvore.fit(X_treino, y_treino)
arvore.classes_